In [7]:
import mlflow
from lung_function.modules.compute_metrics import icc, metrics
from mlflow import log_params

In [28]:
# for new experiments
fold_ex_dt_1 = {0:2521, 1:2522, 2:2531, 3:2544, 4:2545, 'name':''}
fold_ex_dt_2 = {0:2527, 1:2528, 2:2546, 3:2556, 4:2566, 'name':''}
fold_ex_dt_3 = {0:2529, 1:2530, 2:2547, 3:2557, 4:2567, 'name':''}
fold_ex_dt_4 = {0:2532, 1:2533, 2:2548, 3:2558, 4:2579, 'name':''}

fold_ex_dt_5 = {0:2534, 1:2535, 2:2553, 3:2563, 4:2588, 'name':''}
fold_ex_dt_6 = {0:2536, 1:2537, 2:2552, 3:2562, 4:2587, 'name':''}
fold_ex_dt_7 = {0:2538, 1:2539, 2:2551, 3:2561, 4:2586, 'name':''}
fold_ex_dt_8 = {0:2540, 1:2541, 2:2549, 3:2559, 4:2578, 'name':''}

fold_ex_dt_a = {0:2542, 1:2543, 2:2550, 3:2560, 4:2581, 'name':''}
fold_ex_dt_b = {0:2568, 1:2569, 2:2580, 3:2590, 4:2599, 'name':''}
fold_ex_dt_c = {0:2570, 1:2571, 2:2582, 3:2589, 4:2597, 'name':''}
fold_ex_dt_d = {0:2572, 1:2573, 2:2583, 3:2592, 4:2598, 'name':''}

fold_ex_dt_e = {0:2574, 1:2575, 2:2585, 3:2593, 4:2602, 'name':''}
fold_ex_dt_f = {0:2576, 1:2577, 2:2584, 3:2591, 4:2596, 'name':''}
fold_ex_dt_g = {0:2594, 1:2595, 2:2604, 3:2612, 4:2616, 'name':''}
fold_ex_dt_h = {0:2600, 1:2601, 2:2603, 3:2609, 4:2613, 'name':''}

fold_ex_dt_i = {0:2605, 1:2606, 2:2610, 3:2614, 4:2617, 'name':''}
fold_ex_dt_j = {0:2607, 1:2608, 2:2611, 3:2615, 4:2618, 'name':''}

fold_ex_dt_k = {0:902, 1:903, 2:915, 3:921, 4:926, 'name':''}
fold_ex_dt_l = {0:904, 1:905, 2:914, 3:919, 4:924, 'name':''}
fold_ex_dt_m = {0:906, 1:907, 2:918, 3:925, 4:930, 'name':''}
fold_ex_dt_n = {0:908, 1:909, 2:916, 3:923, 4:929, 'name':''}
fold_ex_dt_o = {0:910, 1:911, 2:920, 3:927, 4:931, 'name':''}
fold_ex_dt_p = {0:912, 1:913, 2:917, 3:922, 4:928, 'name':''}

fold_ex_dt_k = {0:2258, 1:2259, 2:2269, 3:2271, 4:2272, 'name':''}
fold_ex_dt_l = {0:2140, 1:2141, 2:2162, 3:2175, 4:2191, 'name':''}
fold_ex_dt_m = {0:2138, 1:2139, 2:2160, 3:2173, 4:2190, 'name':''}
fold_ex_dt_n = {0:2148, 1:2149, 2:2159, 3:2172, 4:2189, 'name':''}
fold_ex_dt_o = {0:2144, 1:2145, 2:2158, 3:2171, 4:2185, 'name':''}
fold_ex_dt_p = {0:2142, 1:2143, 2:2154, 3:2167, 4:2180, 'name':''}

fold_ex_dt_q = {0:2194, 1:2195, 2:2249, 3:2262, 4:2270, 'name':''}
fold_ex_dt_q = {0:429, 1:430, 2:432, 3:434, 4:436, 'name':''}

fold_ex_dt_ls = [fold_ex_dt_q]

In [29]:
mlflow.set_tracking_uri("http://nodelogin02:5000")
experiment = mlflow.set_experiment("lung_fun_db15")

# Create a run under the default experiment (whose id is '0').
client = mlflow.MlflowClient()
parent_dir = '/home/jjia/data/lung_function/lung_function/scripts/results/experiments/'

for i in fold_ex_dt_ls:
    experiment_id = i[0]
    run_ls = client.search_runs(experiment_ids=[experiment.experiment_id], run_view_type=1, filter_string=f"params.id LIKE '%{experiment_id}%'")
    if len(run_ls)!=0:
        run_ = run_ls[-1]
        run_id = run_.info.run_id
        
        
        with mlflow.start_run(run_id=run_id):
            label_fpath = parent_dir + str(i[1]) + '/test_label.csv'
            pred_fpath = parent_dir + str(i[0]) + '/test_pred.csv'
            
            # add icc
            icc_value = icc(label_fpath, pred_fpath, ignore_1st_column=True)
            icc_value_ensemble = {'ensumble_' + k:v  for k, v in icc_value.items()}  # update keys
            print(icc_value_ensemble)
            log_params(icc_value_ensemble)
            # print(run_id)
            
            # add r
            r_p_value = metrics(pred_fpath, label_fpath, ignore_1st_column=True)
            r_p_value_ensemble = {'ensumble_' + k:v  for k, v in r_p_value.items()}  # update keys
            log_params(r_p_value_ensemble)
            
            
            






{'ensumble_icc_test_FVC': 0.61, 'ensumble_icc_test_DLCOc_SB': 0.46, 'ensumble_icc_test_FEV1': 0.53, 'ensumble_icc_test_TLC_He': 0.62}
len_df_label 62
FVC linear regression m, b: 0.5086551425719276 1.227912066398755
FVC linear regression m, b, r^2: 0.5086551425719273 1.227912066398756 0.6937002454331939
ave_mae for FVC is 0.7096774193548387
std_mae for FVC is 0.6571467350794633
mean for FVC is -0.6129032258064516
std for FVC is 0.7482202261124647
Finish plot of  FVC
DLCOc_SB linear regression m, b: 0.34422881691619955 2.9086032912668616
DLCOc_SB linear regression m, b, r^2: 0.34422881691619933 2.9086032912668642 0.47109362318072623
ave_mae for DLCOc_SB is 1.3870967741935485
std_mae for DLCOc_SB is 1.3953244357343335
mean for DLCOc_SB is -1.096774193548387
std for DLCOc_SB is 1.6334179227318921
Finish plot of  DLCOc_SB
FEV1 linear regression m, b: 0.4339399438550754 1.1451162944917304
FEV1 linear regression m, b, r^2: 0.43393994385507556 1.1451162944917312 0.5647832136278403
ave_mae for 